In [ ]:
#import 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
import csv

In [ ]:
#launch url
url = "https://jobs.citi.com/search-jobs"

# create a new Firefox session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)

#cookie_button = driver.find_element_by_id('gdpr-button')
#cookie_button.click()

In [ ]:
python_button = 0
url_list = []

for page in range(1,199):
    soup_level1=BeautifulSoup(driver.page_source, 'lxml')
    
    for link in soup_level1.find_all( 'a', href=re.compile("/job//*")):
        url_list.append(link.get('href'))
        
    next_button = driver.find_element_by_xpath("//a[@class ='next']")
    next_button.click()
    time.sleep(10)
    
driver.quit()

In [ ]:
with open('citigroup_jobs.csv', mode='w') as csv_file:
    fieldnames = ['job_url', 'title', 'job_id', 'location', 'description']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for job_url in url_list:
        company_url = "https://jobs.citi.com"
        visit_url = company_url + job_url
        driver = webdriver.Chrome()
        driver.get(visit_url)
        soup =BeautifulSoup(driver.page_source, 'lxml')
        
        title = soup.find('h2', itemprop = True)
        print("Title",title.get_text())
        
        job_id = soup.find('span', {'class':'job-description2 job-info'})
        job_id_lvl1 = job_id.contents[1]
        job_id_lvl2 = job_id_lvl1.contents[7]
        
        location_lvl1 = job_id.contents[1]
        location = location_lvl1.contents[0]
        
        job_function_lvl1 = job_id.contents[1]
        job_function = job_function_lvl1.contents[2]
        
        schedule_lvl1 = job_id.contents[1]
        schedule = job_function_lvl1.contents[3]
        
        desc = soup.find('div', {'class':'ats-description'})
          
        writer.writerow({'job_url':visit_url, 'title':title.get_text(), 'job_id':job_id_lvl2.contents[1], 'job_function':job_function.contents[1], schedule':schedule.contents[1], 'location':location.contents[1], 'description':desc.get_text()})
        driver.quit()